In [68]:
import pandas as pd
import tensorflow as tf 

train_data = pd.read_csv('/code/mnist/train.csv')
test_data = pd.read_csv('/code/mnist/test.csv')

X = train_data.drop('label', axis=1)
Y = train_data['label']

In [69]:
X = X.astype('float32') / 255
test_data = test_data.astype('float32') / 255

Y = tf.keras.utils.to_categorical(Y, 10)

In [70]:
train_x = X[:round(X.shape[0] * 0.8)]
check_x = X[round(X.shape[0] * 0.8):]

train_y = Y[:round(Y.shape[0] * 0.8)]
check_y = Y[round(Y.shape[0] * 0.8):]

In [71]:
# from tensorflow.keras.layers import Dense, Dropout

# inputs = tf.keras.Input(shape=(784,))
# x_1 = Dense(256, activation='relu')(inputs)
# x_2 = Dropout(0.2)(x_1)
# outputs = Dense(10, activation='softmax')(x_2)

# model = tf.keras.Model(inputs=inputs, outputs=outputs, name='mnist_model')

In [72]:
train_x = train_x.values.reshape(-1, 28, 28, 1)
check_x = check_x.values.reshape(-1, 28, 28, 1)

In [73]:
train_x.shape

(33600, 28, 28, 1)

In [74]:
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten


inputs = tf.keras.Input(shape=(28, 28, 1))
x_1 = Conv2D(32, kernel_size=(5, 5), strides=1, activation='relu', padding='same')(inputs)
x_2 = MaxPooling2D(pool_size=(2, 2), strides=2, padding='same')(x_1)
x_3 = Flatten()(x_2)
x_4 = Dropout(0.4)(x_3)
outputs = Dense(10, activation='softmax')(x_4)

model = tf.keras.Model(inputs=inputs, outputs=outputs, name='mnist_model')

In [75]:
LEARNING_RATE = 0.001
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.categorical_crossentropy,
    metrics=[tf.keras.metrics.categorical_accuracy],
)

In [76]:
train_y.shape

(33600, 10)

In [77]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        patience=2, 
        restore_best_weights=True
    ),
    tf.keras.callbacks.ModelCheckpoint(
        '/code/mnist/data/temp/mnist_sequential_{epoch:03d}_{val_loss:.4f}.h5',
        save_best_only=True
    ),
    tf.keras.callbacks.TensorBoard(
        log_dir='/code/mnist/logs',
        histogram_freq=1
    ),
]

BATCH_SIZE = 128
EPOCHS = 20
VALIDATION_SPLIT = 0.2

history = model.fit(train_x, train_y, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_split=VALIDATION_SPLIT, callbacks=callbacks)
model.save('/code/mnist/mnist_model.h5')

Epoch 1/20


210/210 [==============================] - 4s 19ms/step - loss: 0.5055 - categorical_accuracy: 0.8576 - val_loss: 0.2405 - val_categorical_accuracy: 0.9305
Epoch 2/20
  7/210 [>.............................] - ETA: 3s - loss: 0.2437 - categorical_accuracy: 0.9330

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


210/210 [==============================] - 4s 17ms/step - loss: 0.1986 - categorical_accuracy: 0.9432 - val_loss: 0.1481 - val_categorical_accuracy: 0.9571
Epoch 3/20
210/210 [==============================] - 3s 16ms/step - loss: 0.1361 - categorical_accuracy: 0.9606 - val_loss: 0.1144 - val_categorical_accuracy: 0.9668
Epoch 4/20
210/210 [==============================] - 3s 16ms/step - loss: 0.1104 - categorical_accuracy: 0.9678 - val_loss: 0.0962 - val_categorical_accuracy: 0.9734
Epoch 5/20
210/210 [==============================] - 4s 17ms/step - loss: 0.0933 - categorical_accuracy: 0.9722 - val_loss: 0.0900 - val_categorical_accuracy: 0.9759
Epoch 6/20
210/210 [==============================] - 3s 17ms/step - loss: 0.0814 - categorical_accuracy: 0.9750 - val_loss: 0.0801 - val_categorical_accuracy: 0.9783
Epoch 7/20
210/210 [==============================] - 4s 17ms/step - loss: 0.0744 - categorical_accuracy: 0.9780 - val_loss: 0.0787 - val_categorical_accuracy: 0.9775
Epoch 8/2

In [78]:
model.summary()

Model: "mnist_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_6 (Conv2D)           (None, 28, 28, 32)        832       
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 14, 14, 32)        0         
 g2D)                                                            
                                                                 
 flatten_6 (Flatten)         (None, 6272)              0         
                                                                 
 dropout_6 (Dropout)         (None, 6272)              0         
                                                                 
 dense_6 (Dense)             (None, 10)                62730     
                                                       

In [79]:
model.evaluate(check_x, check_y)

263/263 [==============================] - 0s 1ms/step - loss: 0.0513 - categorical_accuracy: 0.9840


[0.05126721039414406, 0.9840475916862488]

- 0.9811905026435852
- 0.98416668176651 maxpooling
- 0.9783333539962769 karnel(5,5) => (3,3) & padding 'same'
- 0.9825000166893005 karnel => (3,3)
- 0.9840475916862488 drop 0.2 => 0.4